In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D,  concatenate, Conv2DTranspose, Dropout

def UNet(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS):

    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = inputs

    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

    #Expansive path
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()

    return model


In [ ]:
def repeat_elem(tensor, rep):

     return layers.Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=3),
                          arguments={'repnum': rep})(tensor)


def gating_signal(input, out_size, batch_norm=False):

    x = layers.Conv2D(out_size, (1, 1), padding='same')(input)
    if batch_norm:
        x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x

def attention_block(x, gating, inter_shape):
    shape_x = K.int_shape(x)
    shape_g = K.int_shape(gating)

# Getting the x signal to the same shape as the gating signal
    theta_x = layers.Conv2D(inter_shape, (2, 2), strides=(2, 2), padding='same')(x)  # 16
    shape_theta_x = K.int_shape(theta_x)

# Getting the gating signal to the same number of filters as the inter_shape
    phi_g = layers.Conv2D(inter_shape, (1, 1), padding='same')(gating)
    upsample_g = layers.Conv2DTranspose(inter_shape, (3, 3),
                                 strides=(shape_theta_x[1] // shape_g[1], shape_theta_x[2] // shape_g[2]),
                                 padding='same')(phi_g)  # 16

    concat_xg = layers.add([upsample_g, theta_x])
    act_xg = layers.Activation('relu')(concat_xg)
    psi = layers.Conv2D(1, (1, 1), padding='same')(act_xg)
    sigmoid_xg = layers.Activation('sigmoid')(psi)
    shape_sigmoid = K.int_shape(sigmoid_xg)
    upsample_psi = layers.UpSampling2D(size=(shape_x[1] // shape_sigmoid[1], shape_x[2] // shape_sigmoid[2]))(sigmoid_xg)  # 32

    upsample_psi = repeat_elem(upsample_psi, shape_x[3])

    y = layers.multiply([upsample_psi, x])

    result = layers.Conv2D(shape_x[3], (1, 1), padding='same')(y)
    result_bn = layers.BatchNormalization()(result)
    return result_bn

def conv_block(x, filter_size, size, dropout, batch_norm=False):

    conv = layers.Conv2D(size, (filter_size, filter_size), padding="same")(x)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)

    conv = layers.Conv2D(size, (filter_size, filter_size), padding="same")(conv)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)

    if dropout > 0:
        conv = layers.Dropout(dropout)(conv)

    return conv





In [ ]:
from tensorflow.keras import models, layers

def Attention_UNet(input_shape, NUM_CLASSES=1, dropout_rate=0.0, batch_norm=True):
    '''
    Attention UNet,
    Attention_UNet
    '''
    # network structure
    FILTER_NUM = 64 # number of basic filters for the first layer
    FILTER_SIZE = 3 # size of the convolutional filter
    UP_SAMP_SIZE = 2 # size of upsampling filters

    inputs = layers.Input(input_shape, dtype=tf.float32)

    # Downsampling layers
    # DownRes 1, convolution + pooling
    conv_128 = conv_block(inputs, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)
    pool_64 = layers.MaxPooling2D(pool_size=(2,2))(conv_128)
    # DownRes 2
    conv_64 = conv_block(pool_64, FILTER_SIZE, 2*FILTER_NUM, dropout_rate, batch_norm)
    pool_32 = layers.MaxPooling2D(pool_size=(2,2))(conv_64)
    # DownRes 3
    conv_32 = conv_block(pool_32, FILTER_SIZE, 4*FILTER_NUM, dropout_rate, batch_norm)
    pool_16 = layers.MaxPooling2D(pool_size=(2,2))(conv_32)
    # DownRes 4
    conv_16 = conv_block(pool_16, FILTER_SIZE, 8*FILTER_NUM, dropout_rate, batch_norm)
    pool_8 = layers.MaxPooling2D(pool_size=(2,2))(conv_16)
    # DownRes 5, convolution only
    conv_8 = conv_block(pool_8, FILTER_SIZE, 16*FILTER_NUM, dropout_rate, batch_norm)

    # Upsampling layers
    # UpRes 6, attention gated concatenation + upsampling + double residual convolution
    gating_16 = gating_signal(conv_8, 8*FILTER_NUM, batch_norm)
    att_16 = attention_block(conv_16, gating_16, 8*FILTER_NUM)
    up_16 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(conv_8)
    up_16 = layers.concatenate([up_16, att_16], axis=3)
    up_conv_16 = conv_block(up_16, FILTER_SIZE, 8*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 7
    gating_32 = gating_signal(up_conv_16, 4*FILTER_NUM, batch_norm)
    att_32 = attention_block(conv_32, gating_32, 4*FILTER_NUM)
    up_32 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_16)
    up_32 = layers.concatenate([up_32, att_32], axis=3)
    up_conv_32 = conv_block(up_32, FILTER_SIZE, 4*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 8
    gating_64 = gating_signal(up_conv_32, 2*FILTER_NUM, batch_norm)
    att_64 = attention_block(conv_64, gating_64, 2*FILTER_NUM)
    up_64 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_32)
    up_64 = layers.concatenate([up_64, att_64], axis=3)
    up_conv_64 = conv_block(up_64, FILTER_SIZE, 2*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 9
    gating_128 = gating_signal(up_conv_64, FILTER_NUM, batch_norm)
    att_128 = attention_block(conv_128, gating_128, FILTER_NUM)
    up_128 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_64)
    up_128 = layers.concatenate([up_128, att_128], axis=3)
    up_conv_128 = conv_block(up_128, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)

    # 1*1 convolutional layers
    conv_final = layers.Conv2D(NUM_CLASSES, kernel_size=(1,1))(up_conv_128)
    conv_final = layers.BatchNormalization(axis=3)(conv_final)
    conv_final = layers.Activation('sigmoid')(conv_final)  #Change to softmax for multichannel

    # Model integration
    model = models.Model(inputs, conv_final, name="Attention_UNet")
    return model


In [ ]:
def res_conv_block(x, filter_size, size, dropout, batch_norm=False):


    conv = layers.Conv2D(size, (filter_size, filter_size), padding='same')(x)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation('relu')(conv)

    conv = layers.Conv2D(size, (filter_size, filter_size), padding='same')(conv)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    #conv = layers.Activation('relu')(conv)    #Activation before addition with shortcut
    if dropout > 0:
        conv = layers.Dropout(dropout)(conv)

    shortcut = layers.Conv2D(size, kernel_size=(1, 1), padding='same')(x)
    if batch_norm is True:
        shortcut = layers.BatchNormalization(axis=3)(shortcut)

    res_path = layers.add([shortcut, conv])
    res_path = layers.Activation('relu')(res_path)    #Activation after addition with shortcut (Original residual block)
    return res_path


In [ ]:
from tensorflow.keras import models, layers

def Res_UNet(input_shape, NUM_CLASSES=1, dropout_rate=0.0, batch_norm=True):

    # network structure
    FILTER_NUM = 64 # number of filters for the first layer
    FILTER_SIZE = 3 # size of the convolutional filter
    UP_SAMP_SIZE = 2 # size of upsampling filters


    inputs = layers.Input(input_shape, dtype=tf.float32)

    # Downsampling layers
    # DownRes 1, convolution + pooling
    conv_128 = res_conv_block(inputs, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)
    pool_64 = layers.MaxPooling2D(pool_size=(2,2))(conv_128)

    # DownRes 2
    conv_64 = res_conv_block(pool_64, FILTER_SIZE, 2*FILTER_NUM, dropout_rate, batch_norm)
    pool_32 = layers.MaxPooling2D(pool_size=(2,2))(conv_64)
    # DownRes 3
    conv_32 = res_conv_block(pool_32, FILTER_SIZE, 4*FILTER_NUM, dropout_rate, batch_norm)
    pool_16 = layers.MaxPooling2D(pool_size=(2,2))(conv_32)
    # DownRes 4
    conv_16 = res_conv_block(pool_16, FILTER_SIZE, 8*FILTER_NUM, dropout_rate, batch_norm)
    pool_8 = layers.MaxPooling2D(pool_size=(2,2))(conv_16)
    # DownRes 5, convolution only
    conv_8 = res_conv_block(pool_8, FILTER_SIZE, 16*FILTER_NUM, dropout_rate, batch_norm)

    # Upsampling layers

    up_16 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(conv_8)
    up_16 = layers.concatenate([up_16, conv_16], axis=3)
    up_conv_16 = res_conv_block(up_16, FILTER_SIZE, 8*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 7

    up_32 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_16)
    up_32 = layers.concatenate([up_32, conv_32], axis=3)
    up_conv_32 = res_conv_block(up_32, FILTER_SIZE, 4*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 8

    up_64 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_32)
    up_64 = layers.concatenate([up_64, conv_64], axis=3)
    up_conv_64 = res_conv_block(up_64, FILTER_SIZE, 2*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 9

    up_128 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_64)
    up_128 = layers.concatenate([up_128, conv_128], axis=3)
    up_conv_128 = res_conv_block(up_128, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)

    # 1*1 convolutional layers

    conv_final = layers.Conv2D(NUM_CLASSES, kernel_size=(1,1))(up_conv_128)
    conv_final = layers.BatchNormalization(axis=3)(conv_final)
    conv_final = layers.Activation('sigmoid')(conv_final)  #Change to softmax for multichannel

    # Model
    model = models.Model(inputs, conv_final, name="res_UNet")

    return model


In [ ]:
from tensorflow.keras import models, layers

def Att_Res_UNet(input_shape, NUM_CLASSES=1, dropout_rate=0.0, batch_norm=True):
    '''
    Rsidual UNet, with attention

    '''
    # network structure
    FILTER_NUM = 64 # number of basic filters for the first layer
    FILTER_SIZE = 3 # size of the convolutional filter
    UP_SAMP_SIZE = 2 # size of upsampling filters
    # input data
    # dimension of the image depth
    inputs = layers.Input(input_shape, dtype=tf.float32)
    axis = 3

    # Downsampling layers
    # DownRes 1, double residual convolution + pooling
    conv_128 = res_conv_block(inputs, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)
    pool_64 = layers.MaxPooling2D(pool_size=(2,2))(conv_128)
    # DownRes 2
    conv_64 = res_conv_block(pool_64, FILTER_SIZE, 2*FILTER_NUM, dropout_rate, batch_norm)
    pool_32 = layers.MaxPooling2D(pool_size=(2,2))(conv_64)
    # DownRes 3
    conv_32 = res_conv_block(pool_32, FILTER_SIZE, 4*FILTER_NUM, dropout_rate, batch_norm)
    pool_16 = layers.MaxPooling2D(pool_size=(2,2))(conv_32)
    # DownRes 4
    conv_16 = res_conv_block(pool_16, FILTER_SIZE, 8*FILTER_NUM, dropout_rate, batch_norm)
    pool_8 = layers.MaxPooling2D(pool_size=(2,2))(conv_16)
    # DownRes 5, convolution only
    conv_8 = res_conv_block(pool_8, FILTER_SIZE, 16*FILTER_NUM, dropout_rate, batch_norm)

    # Upsampling layers
    # UpRes 6, attention gated concatenation + upsampling + double residual convolution
    gating_16 = gating_signal(conv_8, 8*FILTER_NUM, batch_norm)
    att_16 = attention_block(conv_16, gating_16, 8*FILTER_NUM)
    up_16 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(conv_8)
    up_16 = layers.concatenate([up_16, att_16], axis=axis)
    up_conv_16 = res_conv_block(up_16, FILTER_SIZE, 8*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 7
    gating_32 = gating_signal(up_conv_16, 4*FILTER_NUM, batch_norm)
    att_32 = attention_block(conv_32, gating_32, 4*FILTER_NUM)
    up_32 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_16)
    up_32 = layers.concatenate([up_32, att_32], axis=axis)
    up_conv_32 = res_conv_block(up_32, FILTER_SIZE, 4*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 8
    gating_64 = gating_signal(up_conv_32, 2*FILTER_NUM, batch_norm)
    att_64 = attention_block(conv_64, gating_64, 2*FILTER_NUM)
    up_64 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_32)
    up_64 = layers.concatenate([up_64, att_64], axis=axis)
    up_conv_64 = res_conv_block(up_64, FILTER_SIZE, 2*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 9
    gating_128 = gating_signal(up_conv_64, FILTER_NUM, batch_norm)
    att_128 = attention_block(conv_128, gating_128, FILTER_NUM)
    up_128 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_64)
    up_128 = layers.concatenate([up_128, att_128], axis=axis)
    up_conv_128 = res_conv_block(up_128, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)

    # 1*1 convolutional layers

    conv_final = layers.Conv2D(NUM_CLASSES, kernel_size=(1,1))(up_conv_128)
    conv_final = layers.BatchNormalization(axis=axis)(conv_final)
    conv_final = layers.Activation('sigmoid')(conv_final)  #Change to softmax for multichannel

    # Model integration
    model = models.Model(inputs, conv_final, name="AttentionResUNet")
    return model


In [ ]:
def fractal_block(x, filter_size, size, dropout, batch_norm=False):

    conv_a1 = layers.Conv2D(size, (filter_size, filter_size), padding='same')(x)
    if batch_norm is True:
        conv_a1 = layers.BatchNormalization(axis=3)(conv_a1)
    conv_a1 = layers.Activation('relu')(conv_a1)

    conv_a1 = layers.Conv2D(size, (filter_size, filter_size), padding='same')(conv_a1)
    if batch_norm is True:
        conv_a1 = layers.BatchNormalization(axis=3)(conv_a1)
    conv_a1 = layers.Activation('relu')(conv_a1)    #Activation before addition with shortcut
    if dropout > 0:
        conv_a1 = layers.Dropout(dropout)(conv_a1)


    conv_a2 = layers.Conv2D(size, (filter_size, filter_size), padding='same')(x)
    if batch_norm is True:
        conv_a2 = layers.BatchNormalization(axis=3)(conv_a2)
    conv_a2 = layers.Activation('relu')(conv_a2)

    if dropout > 0:
        conv_a2 = layers.Dropout(dropout)(conv_a2)


    conv_a3 = layers.Conv2D(size, (filter_size, filter_size), padding='same')(x)
    if batch_norm is True:
        conv_a3 = layers.BatchNormalization(axis=3)(conv_a3)
    conv_a3 = layers.Activation('relu')(conv_a3)

    if dropout > 0:
        conv_a3 = layers.Dropout(dropout)(conv_a3)

    fractal_join1 = layers.add([conv_a1, conv_a2])
    #fractal_join1 = layers.Activation('relu')(fractal_join1)    #Activation after addition with shortcut (Original residual block)
    x2 = fractal_join1

    conv_a4 = layers.Conv2D(size, (filter_size, filter_size), padding='same')(x2)
    if batch_norm is True:
        conv_a4 = layers.BatchNormalization(axis=3)(conv_a4)
    conv_a4 = layers.Activation('relu')(conv_a4)

    conv_a4 = layers.Conv2D(size, (filter_size, filter_size), padding='same')(conv_a4)
    if batch_norm is True:
        conv_a4 = layers.BatchNormalization(axis=3)(conv_a4)
    conv_a4 = layers.Activation('relu')(conv_a4)    #Activation before addition with shortcut
    if dropout > 0:
        conv_a4 = layers.Dropout(dropout)(conv_a4)

    conv_a5 = layers.Conv2D(size, (filter_size, filter_size), padding='same')(x2)
    if batch_norm is True:
        conv_a5 = layers.BatchNormalization(axis=3)(conv_a5)
    conv_a5 = layers.Activation('relu')(conv_a5)

    if dropout > 0:
        conv_a5 = layers.Dropout(dropout)(conv_a5)

    #fractal_join2 = layers.add([conv_a4, conv_a5, conv_a3])
 



    return fractal_join2


: 

In [ ]:
from tensorflow.keras import models, layers

def FractalSpiNet(input_shape, NUM_CLASSES=1, dropout_rate=0.0, batch_norm=True):

    # network structure
    FILTER_NUM = 64 # number of filters for the first layer
    FILTER_SIZE = 3 # size of the convolutional filter
    UP_SAMP_SIZE = 2 # size of upsampling filters


    inputs = layers.Input(input_shape, dtype=tf.float32)

    # Downsampling layers
    # DownRes 1, convolution + pooling
    conv_128 = fractal_block(inputs, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)
    pool_64 = layers.MaxPooling2D(pool_size=(2,2))(conv_128)

    # DownRes 2
    conv_64 = fractal_block(pool_64, FILTER_SIZE, 2*FILTER_NUM, dropout_rate, batch_norm)
    pool_32 = layers.MaxPooling2D(pool_size=(2,2))(conv_64)
    # DownRes 3
    conv_32 = fractal_block(pool_32, FILTER_SIZE, 4*FILTER_NUM, dropout_rate, batch_norm)
    pool_16 = layers.MaxPooling2D(pool_size=(2,2))(conv_32)
    # DownRes 4
    conv_16 = fractal_block(pool_16, FILTER_SIZE, 8*FILTER_NUM, dropout_rate, batch_norm)
    pool_8 = layers.MaxPooling2D(pool_size=(2,2))(conv_16)
    # DownRes 5, convolution only
    conv_8 = fractal_block(pool_8, FILTER_SIZE, 16*FILTER_NUM, dropout_rate, batch_norm)

    # Upsampling layers

    up_16 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(conv_8)
    up_16 = layers.concatenate([up_16, conv_16], axis=3)
    up_conv_16 = fractal_block(up_16, FILTER_SIZE, 8*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 7

    up_32 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_16)
    up_32 = layers.concatenate([up_32, conv_32], axis=3)
    up_conv_32 = fractal_block(up_32, FILTER_SIZE, 4*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 8

    up_64 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_32)
    up_64 = layers.concatenate([up_64, conv_64], axis=3)
    up_conv_64 = fractal_block(up_64, FILTER_SIZE, 2*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 9

    up_128 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_64)
    up_128 = layers.concatenate([up_128, conv_128], axis=3)
    up_conv_128 = fractal_block(up_128, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)

    # 1*1 convolutional layers

    conv_final = layers.Conv2D(NUM_CLASSES, kernel_size=(1,1))(up_conv_128)
    conv_final = layers.BatchNormalization(axis=3)(conv_final)
    conv_final = layers.Activation('sigmoid')(conv_final)  #Change to softmax for multichannel

    # Model
    model = models.Model(inputs, conv_final, name="FractalSpiNet")

    return model


# DATA INPUT

In [ ]:

image_directory ="/home/sysadmin/RKY/unet/newaug_aksiyal_son_silinmis/image/"
mask_directory = "/home/sysadmin/RKY/unet/newaug_aksiyal_son_silinmis/mask/"


In [ ]:

SIZE = 128
image_dataset = []
mask_dataset = []
images = sorted(os.listdir(image_directory))
for i, image_name in enumerate(images):
    if (image_name.split('.')[1] == 'png'):
        image = cv2.imread(image_directory+image_name, 0)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        image_dataset.append(np.array(image))


masks = sorted(os.listdir(mask_directory))
for i, image_name in enumerate(masks):
    if (image_name.split('.')[1] == 'png'):
        image = cv2.imread(mask_directory+image_name, 0)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        mask_dataset.append(np.array(image))



# NORMALIZATION IMAGES AND MASKS

In [ ]:

image_dataset = np.expand_dims(normalize(np.array(image_dataset)),3)
print(image_dataset[0].shape)
mask_dataset = np.expand_dims((np.array(mask_dataset)),3)/255
print(mask_dataset[0].shape)

(128, 128, 1)
(128, 128, 1)


# MODEL SELCTION

In [ ]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_dataset, mask_dataset, test_size = 0.20, random_state = 0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((862, 128, 128, 1),
 (216, 128, 128, 1),
 (862, 128, 128, 1),
 (216, 128, 128, 1))

# Data Visualization

In [ ]:
import random
import numpy as np
image_number = random.randint(0, len(X_train))
print(image_number)
plt.figure(figsize=(24, 12))
plt.subplot(1,4,1)
plt.axis('off')
plt.imshow(np.reshape(X_train[image_number], (128, 128)), cmap='gray')
plt.subplot(1,4,2)
plt.axis('off')
plt.imshow(np.reshape(y_train[image_number], (128, 128)), cmap='gray')
plt.subplot(1,4,3)
plt.axis('off')
plt.imshow(np.reshape(X_train[image_number], (128, 128)),)
plt.subplot(1,4,4)
plt.axis('off')
plt.imshow(np.reshape(y_train[image_number], (128, 128)),)
plt.show()

NameError: name 'X_train' is not defined

# TEMEL MODEL U-NET

In [ ]:


'''
UNet
'''
unet_model = UNet(input_shape)
unet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


print(unet_model.summary())

start1 = datetime.now()

#weigth_path=os.getcwd()+"/weights_unet/unet-{epoch:04d}.hdf5"
weigth_path="/media/sysadmin/stock/weights_unet/13_unet-{epoch:04d}.hdf5"
cp_callback=ModelCheckpoint(
    weigth_path,verbose=1,save_weights_only=True,
    save_freq="epoch"
)
unet_history = unet_model.fit(X_train, y_train,
                    verbose=1,
                    callbacks=[cp_callback],
                    batch_size = batch_size,
                    validation_data=(X_test, y_test ),
                    shuffle=False,
                    epochs=200)

stop1 = datetime.now()
#Execution time of the model
execution_time_Unet = stop1-start1
print("UNet execution time is: ", execution_time_Unet)

unet_model.save('AksiyalSON_MS_SCI_MAKALE_IEEE_son_UNet13.hdf5')


Unet_model_history_df = pd.DataFrame(unet_history.history)
Unet_model_history_csv_file = 'Unet_AksiyalSON_MS_SCI_MAKALE_IEEE_13.csv'

with open('Unet_AksiyalSON_MS_SCI_MAKALE_IEEE_13.csv', mode='w') as f:
    Unet_model_history_df.to_csv(f)

Model: "UNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d_38 (Conv2D)              (None, 128, 128, 64) 640         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_38 (BatchNo (None, 128, 128, 64) 256         conv2d_38[0][0]                  
__________________________________________________________________________________________________
activation_38 (Activation)      (None, 128, 128, 64) 0           batch_normalization_38[0][0]     
_______________________________________________________________________________________________

108/108 [==============================] - 16s 99ms/step - loss: 0.6356 - accuracy: 0.9606 - val_loss: 0.5176 - val_accuracy: 0.9905

Epoch 00001: saving model to /media/sysadmin/stock/weights_unet/13_unet-0001.hdf5
Epoch 2/200
108/108 [==============================] - 9s 79ms/step - loss: 0.5756 - accuracy: 0.9932 - val_loss: 0.5288 - val_accuracy: 0.9905

Epoch 00002: saving model to /media/sysadmin/stock/weights_unet/13_unet-0002.hdf5
Epoch 3/200
108/108 [==============================] - 9s 80ms/step - loss: 0.5282 - accuracy: 0.9958 - val_loss: 0.4973 - val_accuracy: 0.9905

Epoch 00003: saving model to /media/sysadmin/stock/weights_unet/13_unet-0003.hdf5
Epoch 4/200
108/108 [==============================] - 9s 82ms/step - loss: 0.4856 - accuracy: 0.9966 - val_loss: 0.4764 - val_accuracy: 0.9905

Epoch 00004: saving model to /media/sysadmin/stock/weights_unet/13_unet-0004.hdf5
Epoch 5/200
108/108 [==============================] - 9s 83ms/step - loss: 0.4477 - accuracy: 0.9968 -

108/108 [==============================] - 9s 79ms/step - loss: 0.0567 - accuracy: 0.9980 - val_loss: 0.0568 - val_accuracy: 0.9977

Epoch 00037: saving model to /media/sysadmin/stock/weights_unet/13_unet-0037.hdf5
Epoch 38/200
108/108 [==============================] - 8s 78ms/step - loss: 0.0537 - accuracy: 0.9980 - val_loss: 0.0539 - val_accuracy: 0.9977

Epoch 00038: saving model to /media/sysadmin/stock/weights_unet/13_unet-0038.hdf5
Epoch 39/200
108/108 [==============================] - 8s 79ms/step - loss: 0.0510 - accuracy: 0.9980 - val_loss: 0.0511 - val_accuracy: 0.9975

Epoch 00039: saving model to /media/sysadmin/stock/weights_unet/13_unet-0039.hdf5
Epoch 40/200
108/108 [==============================] - 8s 79ms/step - loss: 0.0527 - accuracy: 0.9975 - val_loss: 0.0852 - val_accuracy: 0.9902

Epoch 00040: saving model to /media/sysadmin/stock/weights_unet/13_unet-0040.hdf5
Epoch 41/200
108/108 [==============================] - 8s 79ms/step - loss: 0.0483 - accuracy: 0.997

108/108 [==============================] - 8s 78ms/step - loss: 0.0108 - accuracy: 0.9981 - val_loss: 0.0132 - val_accuracy: 0.9976

Epoch 00073: saving model to /media/sysadmin/stock/weights_unet/13_unet-0073.hdf5
Epoch 74/200
108/108 [==============================] - 8s 78ms/step - loss: 0.0104 - accuracy: 0.9981 - val_loss: 0.0146 - val_accuracy: 0.9977

Epoch 00074: saving model to /media/sysadmin/stock/weights_unet/13_unet-0074.hdf5
Epoch 75/200
108/108 [==============================] - 8s 78ms/step - loss: 0.0099 - accuracy: 0.9982 - val_loss: 0.0134 - val_accuracy: 0.9978

Epoch 00075: saving model to /media/sysadmin/stock/weights_unet/13_unet-0075.hdf5
Epoch 76/200
108/108 [==============================] - 8s 78ms/step - loss: 0.0095 - accuracy: 0.9981 - val_loss: 0.0125 - val_accuracy: 0.9977

Epoch 00076: saving model to /media/sysadmin/stock/weights_unet/13_unet-0076.hdf5
Epoch 77/200
108/108 [==============================] - 8s 78ms/step - loss: 0.0092 - accuracy: 0.998

108/108 [==============================] - 8s 78ms/step - loss: 0.0032 - accuracy: 0.9982 - val_loss: 0.0060 - val_accuracy: 0.9978

Epoch 00109: saving model to /media/sysadmin/stock/weights_unet/13_unet-0109.hdf5
Epoch 110/200
108/108 [==============================] - 8s 78ms/step - loss: 0.0031 - accuracy: 0.9982 - val_loss: 0.0058 - val_accuracy: 0.9978

Epoch 00110: saving model to /media/sysadmin/stock/weights_unet/13_unet-0110.hdf5
Epoch 111/200
108/108 [==============================] - 8s 78ms/step - loss: 0.0030 - accuracy: 0.9982 - val_loss: 0.0062 - val_accuracy: 0.9978

Epoch 00111: saving model to /media/sysadmin/stock/weights_unet/13_unet-0111.hdf5
Epoch 112/200
108/108 [==============================] - 8s 78ms/step - loss: 0.0029 - accuracy: 0.9982 - val_loss: 0.0065 - val_accuracy: 0.9978

Epoch 00112: saving model to /media/sysadmin/stock/weights_unet/13_unet-0112.hdf5
Epoch 113/200
108/108 [==============================] - 8s 78ms/step - loss: 0.0029 - accuracy: 0

108/108 [==============================] - 8s 78ms/step - loss: 0.0017 - accuracy: 0.9982 - val_loss: 0.0050 - val_accuracy: 0.9978

Epoch 00145: saving model to /media/sysadmin/stock/weights_unet/13_unet-0145.hdf5
Epoch 146/200
108/108 [==============================] - 8s 78ms/step - loss: 0.0017 - accuracy: 0.9982 - val_loss: 0.0047 - val_accuracy: 0.9978

Epoch 00146: saving model to /media/sysadmin/stock/weights_unet/13_unet-0146.hdf5
Epoch 147/200
108/108 [==============================] - 8s 78ms/step - loss: 0.0017 - accuracy: 0.9982 - val_loss: 0.0047 - val_accuracy: 0.9978

Epoch 00147: saving model to /media/sysadmin/stock/weights_unet/13_unet-0147.hdf5
Epoch 148/200
108/108 [==============================] - 8s 78ms/step - loss: 0.0017 - accuracy: 0.9982 - val_loss: 0.0046 - val_accuracy: 0.9978

Epoch 00148: saving model to /media/sysadmin/stock/weights_unet/13_unet-0148.hdf5
Epoch 149/200
108/108 [==============================] - 8s 78ms/step - loss: 0.0016 - accuracy: 0

108/108 [==============================] - 8s 78ms/step - loss: 0.0014 - accuracy: 0.9982 - val_loss: 0.0038 - val_accuracy: 0.9978

Epoch 00181: saving model to /media/sysadmin/stock/weights_unet/13_unet-0181.hdf5
Epoch 182/200
108/108 [==============================] - 8s 78ms/step - loss: 0.0014 - accuracy: 0.9982 - val_loss: 0.0040 - val_accuracy: 0.9978

Epoch 00182: saving model to /media/sysadmin/stock/weights_unet/13_unet-0182.hdf5
Epoch 183/200
108/108 [==============================] - 8s 78ms/step - loss: 0.0014 - accuracy: 0.9982 - val_loss: 0.0042 - val_accuracy: 0.9978

Epoch 00183: saving model to /media/sysadmin/stock/weights_unet/13_unet-0183.hdf5
Epoch 184/200
108/108 [==============================] - 8s 78ms/step - loss: 0.0013 - accuracy: 0.9982 - val_loss: 0.0044 - val_accuracy: 0.9978

Epoch 00184: saving model to /media/sysadmin/stock/weights_unet/13_unet-0184.hdf5
Epoch 185/200
108/108 [==============================] - 8s 78ms/step - loss: 0.0013 - accuracy: 0

In [ ]:
'''
fractal2_UNet
'''
fractal2_unet_model = fractal2_UNet(input_shape)
fractal2_unet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


print(fractal2_unet_model .summary())

start5 = datetime.now()

#weigth_path=os.getcwd()+"/weights_unet/unet-{epoch:04d}.hdf5"
weigth_path="/media/sysadmin/stock/weights_fractal/13_fractalunet-{epoch:04d}.hdf5"
cp_callback=ModelCheckpoint(
    weigth_path,verbose=1,save_weights_only=True,
    save_freq="epoch"
)

fractal2_unet_history = fractal2_unet_model.fit(X_train, y_train,
                    verbose=1,
                    callbacks=[cp_callback],
                    batch_size = batch_size,
                    validation_data=(X_test, y_test ),
                    shuffle=False,
                    epochs=200)




stop5 = datetime.now()
#Execution time of the model
execution_time_fractal2_unet = stop5-start5
print("fractal2 UNet execution time is: ", execution_time_fractal2_unet)

#unet_model.save('mitochondria_UNet_50epochs_B_focal.hdf5')
fractal2_unet_model.save('AKSİYALSON_MS_SCI_MAKALE_IEEE_1_fractal2_unet13.hdf5')


fractal2_unet_history_df = pd.DataFrame(fractal2_unet_history.history)
fractal2_unet_history_csv_file = 'AKSİYALSON_MS_SCI_MAKALE_IEEE_1_fractal2_unet13.csv'

with open('AKSİYALSON_MS_SCI_MAKALE_IEEE_1_fractal2_unet13.csv', mode='w') as f:
    fractal2_unet_history_df.to_csv(f)

In [ ]:
'''
Attention UNet
'''
karma_att_unet_model = Attention_UNet(input_shape)

karma_att_unet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


print(karma_att_unet_model.summary())
start2 = datetime.now()



#weigth_path=os.getcwd()+"/weights_unet/unet-{epoch:04d}.hdf5"
weigth_path="/media/sysadmin/stock/weights_att/13_att_unet-{epoch:04d}.hdf5"
cp_callback=ModelCheckpoint(
    weigth_path,verbose=1,save_weights_only=True,
    save_freq="epoch"
)


karma_att_unet_history = karma_att_unet_model.fit(X_train, y_train,
                    verbose=1,
                    callbacks=[cp_callback],
                    batch_size = batch_size,
                    validation_data=(X_test, y_test ),
                    shuffle=False,
                    epochs=200)

stop2 = datetime.now()
#Execution time of the model
execution_time_karma_Att_Unet = stop2-start2
print("karma_Attention UNet execution time is: ", execution_time_karma_Att_Unet)

#att_unet_model.save('mitochondria_Attention_UNet_50epochs_B_focal.hdf5')
karma_att_unet_model.save('AKSİYALSON_MS_SCI_MAKALE_IEEE_1_karma_att_unet_model-13.hdf5')

karma_att_unet_history_df = pd.DataFrame(karma_att_unet_history.history)
karma_att_unet_history_csv_file = 'AKSİYALSON_MS_SCI_MAKALE_IEEE_1_karma_att_unet_model-13.csv'

with open('AKSİYALSON_MS_SCI_MAKALE_IEEE_1_karma_att_unet_model-13.csv', mode='w') as f:
    karma_att_unet_history_df.to_csv(f)

In [ ]:
'''
res_UNet_UNet
'''
karma_res_UNet_model = res_UNet(input_shape)
karma_res_UNet_model.compile(optimizer=Adam(lr = 1e-2), loss='binary_crossentropy', metrics=['accuracy'])


print(karma_res_UNet_model.summary())

start3 = datetime.now()

#weigth_path=os.getcwd()+"/weights_unet/unet-{epoch:04d}.hdf5"
weigth_path="/media/sysadmin/stock/weights_res/13-resunet-{epoch:04d}.hdf5"
cp_callback=ModelCheckpoint(
    weigth_path,verbose=1,save_weights_only=True,
    save_freq="epoch"
)


karma_res_UNet_history = karma_res_UNet_model.fit(X_train, y_train,
                    verbose=1,
                    callbacks=[cp_callback],
                    batch_size = batch_size,
                    validation_data=(X_test, y_test ),
                    shuffle=False,
                    epochs=200)

stop3 = datetime.now()
#Execution time of the model
execution_time_karma_res_UNet_modelres_UNet = stop3-start3
print("karma_res_UNet_modelres_UNet execution time is: ", execution_time_karma_res_UNet_modelres_UNet)

#unet_model.save('mitochondria_UNet_50epochs_B_focal.hdf5')
karma_res_UNet_model.save('AKSİYALSON_MS_SCI_MAKALE_IEEE_1_karma_res_UNet_model-13.hdf5')

karma_res_UNet_history_df = pd.DataFrame(karma_res_UNet_history.history)
karma_res_UNet_history_csv_file = 'AKSİYALSON_MS_SCI_MAKALE_IEEE_1_karma_res_UNet_model-13.csv'

with open('AKSİYALSON_MS_SCI_MAKALE_IEEE_1_karma_res_UNet_model-13.csv', mode='w') as f:
    karma_res_UNet_history_df.to_csv(f)

In [ ]:
'''
Attention Residual Unet
'''
karma_att_res_unet_model = Attention_ResUNet(input_shape)

karma_att_res_unet_model.compile(optimizer=Adam(lr = 1e-2), loss='binary_crossentropy',
              metrics=['accuracy'])


# att_res_unet_model.compile(optimizer=Adam(lr = 1e-3), loss='binary_crossentropy',
#               metrics=['accuracy', jacard_coef])

print(karma_att_res_unet_model.summary())


start4 = datetime.now()

#weigth_path=os.getcwd()+"/weights_unet/unet-{epoch:04d}.hdf5"
weigth_path="/media/sysadmin/stock/weights_resatt/13_resattunet-{epoch:04d}.hdf5"
cp_callback=ModelCheckpoint(
    weigth_path,verbose=1,save_weights_only=True,
    save_freq="epoch"
)


karma_att_res_unet_history = karma_att_res_unet_model.fit(X_train, y_train,
                    verbose=1,
                    callbacks=[cp_callback],
                    batch_size = batch_size,
                    validation_data=(X_test, y_test ),
                    shuffle=False,
                    epochs=200)
stop4 = datetime.now()

#Execution time of the model
execution_time_karma_att_res = stop4-start4
print("karma att res cution time is: ", execution_time_karma_att_res)

karma_att_res_unet_model.save('AKSİYALSON_MS_SCI_MAKALE_IEEE_1_karma_att_res_unet_model-13.hdf5')


karma_att_res_unet_history_df = pd.DataFrame(karma_att_res_unet_history.history)
karma_att_res_unet_history_csv_file = 'AKSİYALSON_MS_SCI_MAKALE_IEEE_1_karma_att_res_unet_model-13.csv'

with open('AKSİYALSON_MS_SCI_MAKALE_IEEE_1_karma_att_res_unet_model-13.csv', mode='w') as f:
    karma_att_res_unet_history_df.to_csv(f)

In [ ]:
#Evaluate the model

history = unet_history
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss for Spinalcord Dataset')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

#acc = history.history['acc']
acc = history.history['accuracy']
#val_acc = history.history['val_acc']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy for Spinalcord Dataset')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

model = unet_model
	# evaluate model
_, acc = model.evaluate(X_test, y_test)
print("Accuracy = ", (acc * 100.0), "%")

#IOU
model = unet_model
y_pred=model.predict(X_test)
y_pred_thresholded = y_pred > 0.5

intersection = np.logical_and(y_test, y_pred_thresholded)
union = np.logical_or(y_test, y_pred_thresholded)
iou_score = np.sum(intersection) / np.sum(union)
print("IoU socre is: ", iou_score)



NameError: name 'unet_history' is not defined

# model deneme

In [ ]:
import lesionQcMetrics as QcMetric
import Metric_eval as EvMetric


model = unet_model
y_pred=model.predict(X_test)


i=0



toplam_sp_dice=0
toplam_sp_jaccard=0
toplam_sp_ppv=0
toplam_sp_precision=0
toplam_sp_tpr=0
toplam_sp_sensitivity=0
toplam_sp_avd=0
toplam_sp_ravd =0
toplam_sp_assd  =0
toplam_sp_volume_correlation=0
toplam_sp_lfpr=0
toplam_sp_ltpr=0
toplam_sp_recall=0
toplam_sp_specificity=0
toplam_sp_true_negative_rate=0
toplam_sp_true_positive_rate=0
toplam_sp_positive_predictive_value=0
toplam_sp_hd=0
toplam_sp_hd95=0
toplam_sp_asd=0
toplam_sp_volume_change_correlation=0
toplam_sp_obj_assd=0
toplam_sp_obj_asd=0
toplam_sp_obj_tpr=0
toplam_sp_obj_fpr=0


toplam_MS_dice=0
toplam_MS_jaccard=0
toplam_MS_ppv=0
toplam_MS_precision=0
toplam_MS_tpr=0
toplam_MS_sensitivity=0
toplam_MS_avd=0
toplam_MS_ravd =0
toplam_MS_assd =0
toplam_MS_volume_correlation=0
toplam_MS_lfpr=0
toplam_MS_ltpr=0
toplam_MS_recall=0
toplam_MS_specificity=0
toplam_MS_true_negative_rate=0
toplam_MS_true_positive_rate=0
toplam_MS_positive_predictive_value=0
toplam_MS_hd=0
toplam_MS_hd95=0
toplam_MS_asd=0
toplam_MS_volume_change_correlation=0
toplam_MS_obj_assd=0
toplam_MS_obj_asd=0
toplam_MS_obj_tpr=0
toplam_MS_obj_fpr=0



toplam_all_dice=0
toplam_all_jaccard=0
toplam_all_ppv=0
toplam_all_precision=0
toplam_all_tpr=0
toplam_all_sensitivity=0
toplam_all_avd=0
toplam_all_ravd =0
toplam_all_assd =0
toplam_all_volume_correlation=0
toplam_all_lfpr=0
toplam_all_ltpr=0
toplam_all_recall=0
toplam_all_specificity=0
toplam_all_true_negative_rate=0
toplam_all_true_positive_rate=0
toplam_all_positive_predictive_value=0
toplam_all_hd=0
toplam_all_hd95=0
toplam_all_asd=0
toplam_all_volume_change_correlation=0
toplam_all_obj_assd=0
toplam_all_obj_asd=0
toplam_all_obj_tpr=0
toplam_all_obj_fpr=0


for i in range(len(X_test)):
#     image_number = random.randint(0, (len(X_test)-1))
    #image_number=0
    image_number=i

    y_testR=img_as_ubyte(y_test[image_number])

    y_predR=img_as_ubyte(y_pred[image_number])
    #print(y_predR)
    predB=takeSame(y_predR,85,170,128)
    testB=takeSame(y_testR,85,170,128)
    predC=takeSame(y_predR,170,255,255)
    testC=takeSame(y_testR,170,255,255)
    predD=takeSame(y_predR,85,255,255) # tüm segmentasyon tek sınıf
    testD=takeSame(y_testR,85,255,255) # tüm segmentasyon tek sınıf

    """ MS  all metrics """

    MS_dice=EvMetric.dc(predB,testB)
    MS_jaccard=EvMetric.jc(predB,testB)
    MS_precision=EvMetric.precision(predB,testB)
    MS_recall=EvMetric.recall(predB,testB)
    MS_ppv=EvMetric.positive_predictive_value(predB,testB)
    MS_sensitivity=EvMetric.sensitivity(predB,testB)
    MS_specificity=EvMetric.specificity(predB,testB)
    MS_true_negative_rate=EvMetric.true_negative_rate(predB,testB)
    MS_true_positive_rate=EvMetric.true_positive_rate(predB,testB)
    MS_positive_predictive_value=EvMetric.positive_predictive_value(predB,testB)
    MS_hd=EvMetric.hd(predB,testB)
    MS_hd95=EvMetric.hd95(predB,testB)
    MS_assd=EvMetric.assd(predB,testB)
    MS_asd=EvMetric.asd(predB,testB)
    MS_ravd=EvMetric.ravd(predB,testB)
    MS_volume_correlation=EvMetric.volume_correlation(predB,testB)
    MS_volume_change_correlation=EvMetric.volume_change_correlation(predB,testB)
    MS_obj_assd=EvMetric.obj_assd(predB,testB)
    MS_obj_asd=EvMetric.obj_asd(predB,testB)
    MS_obj_fpr=EvMetric.obj_fpr(predB,testB)
    MS_obj_tpr=EvMetric.obj_tpr(predB,testB)


    """ spinal without ms metrics """


    sp_dice=EvMetric.dc(predC,testC)
    sp_jaccard=EvMetric.jc(predC,testC)
    sp_precision=EvMetric.precision(predC,testC)
    sp_recall=EvMetric.recall(predC,testC)
    sp_ppv=EvMetric.positive_predictive_value(predC,testC)
    sp_sensitivity=EvMetric.sensitivity(predC,testC)
    sp_specificity=EvMetric.specificity(predC,testC)
    sp_true_negative_rate=EvMetric.true_negative_rate(predC,testC)
    sp_true_positive_rate=EvMetric.true_positive_rate(predC,testC)
    sp_positive_predictive_value=EvMetric.positive_predictive_value(predC,testC)
    sp_hd=EvMetric.hd(predC,testC)
    sp_hd95=EvMetric.hd95(predC,testC)
    sp_assd=EvMetric.assd(predC,testC)
    sp_asd=EvMetric.asd(predC,testC)
    sp_ravd=EvMetric.ravd(predC,testC)
    sp_volume_correlation=EvMetric.volume_correlation(predC,testC)
    sp_volume_change_correlation=EvMetric.volume_change_correlation(predC,testC)
    sp_obj_assd=EvMetric.obj_assd(predC,testC)
    sp_obj_asd=EvMetric.obj_asd(predC,testC)
    sp_obj_tpr=EvMetric.obj_tpr(predC,testC)
    sp_obj_fpr=EvMetric.obj_fpr(predC,testC)

    """ ALL spinal all metrics """

    all_dice=EvMetric.dc(predD,testD)
    all_jaccard=EvMetric.jc(predD,testD)
    all_precision=EvMetric.precision(predD,testD)
    all_recall=EvMetric.recall(predD,testD)
    all_ppv=EvMetric.positive_predictive_value(predD,testD)
    all_sensitivity=EvMetric.sensitivity(predD,testD)
    all_specificity=EvMetric.specificity(predD,testD)
    all_true_negative_rate=EvMetric.true_negative_rate(predD,testD)
    all_true_positive_rate=EvMetric.true_positive_rate(predD,testD)
    all_positive_predictive_value=EvMetric.positive_predictive_value(predD,testD)
    all_hd=EvMetric.hd(predD,testD)
    all_hd95=EvMetric.hd95(predD,testD)
    all_assd=EvMetric.assd(predD,testD)
    all_asd=EvMetric.asd(predD,testD)
    all_ravd=EvMetric.ravd(predD,testD)
    all_volume_correlation=EvMetric.volume_correlation(predD,testD)
    all_volume_change_correlation=EvMetric.volume_change_correlation(predD,testD)
    all_obj_assd=EvMetric.obj_assd(predD,testD)
    all_obj_asd=EvMetric.obj_asd(predD,testD)
    all_obj_tpr=EvMetric.obj_tpr(predD,testD)
    all_obj_fpr=EvMetric.obj_fpr(predD,testD)

    "toplam spinal hesaplama ve ortalama hespalama "
    toplam_sp_dice=toplam_sp_dice+sp_dice
    toplam_sp_jaccard=toplam_sp_jaccard+sp_jaccard
    toplam_sp_precision=toplam_sp_precision+sp_precision
    toplam_sp_recall=toplam_sp_recall+sp_recall
    toplam_sp_ppv=toplam_sp_ppv+sp_ppv
    toplam_sp_sensitivity=toplam_sp_sensitivity+sp_sensitivity
    toplam_sp_specificity=toplam_sp_specificity+sp_specificity
    toplam_sp_true_negative_rate=toplam_sp_true_negative_rate+sp_true_negative_rate
    toplam_sp_true_positive_rate=toplam_sp_true_positive_rate+sp_true_positive_rate
    toplam_sp_positive_predictive_value=toplam_sp_positive_predictive_value+sp_positive_predictive_value
    toplam_sp_hd=toplam_sp_hd+sp_hd
    toplam_sp_hd95=toplam_sp_hd95+sp_hd95
    toplam_sp_assd=toplam_sp_assd+sp_assd
    toplam_sp_asd=toplam_sp_asd+sp_asd
    toplam_sp_ravd=toplam_sp_ravd+sp_ravd
    toplam_sp_obj_assd=toplam_sp_obj_assd+sp_obj_assd
    toplam_sp_obj_asd=toplam_sp_obj_asd+sp_obj_asd
    toplam_sp_obj_tpr=toplam_sp_obj_tpr+sp_obj_tpr
    toplam_sp_obj_fpr=toplam_sp_obj_fpr+sp_obj_fpr


    "toplam ms hesaplama ve ortalama hespalama "
    toplam_MS_dice=toplam_MS_dice+MS_dice
    toplam_MS_jaccard=toplam_MS_jaccard+MS_jaccard
    toplam_MS_precision=toplam_MS_precision+MS_precision
    toplam_MS_recall=toplam_MS_recall+MS_recall
    toplam_MS_ppv=toplam_MS_ppv+MS_ppv
    toplam_MS_sensitivity=toplam_MS_sensitivity+MS_sensitivity
    toplam_MS_specificity=toplam_MS_specificity+MS_specificity
    toplam_MS_true_negative_rate=toplam_MS_true_negative_rate+MS_true_negative_rate
    toplam_MS_true_positive_rate=toplam_MS_true_positive_rate+MS_true_positive_rate
    toplam_MS_positive_predictive_value=toplam_MS_positive_predictive_value+MS_positive_predictive_value
    toplam_MS_hd=toplam_MS_hd+MS_hd
    toplam_MS_hd95=toplam_MS_hd95+MS_hd95
    toplam_MS_assd=toplam_MS_assd+MS_assd
    toplam_MS_asd=toplam_MS_asd+MS_asd
    toplam_MS_ravd=toplam_MS_ravd+MS_ravd
    toplam_MS_obj_assd=toplam_MS_obj_assd+MS_obj_assd
    toplam_MS_obj_asd=toplam_MS_obj_asd+MS_obj_asd
    toplam_MS_obj_tpr=toplam_MS_obj_tpr+MS_obj_tpr
    toplam_MS_obj_fpr=toplam_MS_obj_fpr+MS_obj_fpr

    "toplam ALL spinal hesaplama ve ortalama hespalama "
    #print("Dice Coefficient 2 is: ", dice2)
    #print("Dice Coefficient ort is: ", dice_coefficient_ort2)

    toplam_all_dice=toplam_all_dice+all_dice
    toplam_all_jaccard=toplam_all_jaccard+all_jaccard
    toplam_all_precision=toplam_all_precision+all_precision
    toplam_all_recall=toplam_all_recall+all_recall
    toplam_all_ppv=toplam_all_ppv+all_ppv
    toplam_all_sensitivity=toplam_all_sensitivity+all_sensitivity
    toplam_all_specificity=toplam_all_specificity+all_specificity
    toplam_all_true_negative_rate=toplam_all_true_negative_rate+all_true_negative_rate
    toplam_all_true_positive_rate=toplam_all_true_positive_rate+all_true_positive_rate
    toplam_all_positive_predictive_value=toplam_all_positive_predictive_value+all_positive_predictive_value
    toplam_all_hd=toplam_all_hd+all_hd
    toplam_all_hd95=toplam_all_hd95+all_hd95
    toplam_all_assd=toplam_all_assd+all_assd
    toplam_all_asd=toplam_all_asd+all_asd
    toplam_all_ravd=toplam_all_ravd+all_ravd
    toplam_all_obj_assd=toplam_all_obj_assd+all_obj_assd
    toplam_all_obj_asd=toplam_all_obj_asd+all_obj_asd
    toplam_all_obj_tpr=toplam_all_obj_tpr+all_obj_tpr
    toplam_all_obj_fpr=toplam_all_obj_fpr+all_obj_fpr

    print(image_number)
    #fig.tight_layout(pad=0.2)
    plt.figure(figsize=(24,6))
    plt.subplot(1,6,1)
    plt.title('Image')
    plt.imshow(np.reshape(X_test[image_number], (128, 128)), cmap='gray')
    plt.axis('off')
    plt.subplot(1,6,2)
    plt.title('Mask')
    plt.axis('off')
    plt.imshow(np.reshape(y_test[image_number], (128, 128)), cmap='gray')
    plt.subplot(1,6,3)
    plt.title('Spinal Predict with Ms')
    plt.axis('off')
    plt.imshow(np.reshape(y_predR, (128, 128)), cmap='gray')
    plt.subplot(1,6,4)
    plt.title('Spinal Predict without Ms')
    plt.axis('off')
    plt.imshow(np.reshape(predC, (128, 128)), cmap='gray')
    plt.subplot(1,6,5)
    plt.title('Only Ms Predict')
    plt.axis('off')
    plt.imshow(np.reshape(predB, (128, 128)), cmap='gray')
    plt.subplot(1,6,6)
    plt.title('ALL Spinal Predict')
    plt.axis('off')
    plt.imshow(np.reshape(predD, (128, 128)), cmap='gray')

    plt.show()



    print()
    print("spinal Dice,Jaccard, precision,recall,ppv,sensitivity,specificity: ")
    print("spinal", sp_dice,sp_jaccard, sp_precision, sp_recall,sp_ppv,sp_sensitivity,sp_specificity)
    print()
    print("spinal true_negative_rate, true_positive_rate,sp_positive_predictive_value: ")
    print("spinal",sp_true_negative_rate,sp_true_positive_rate,sp_positive_predictive_value)
    print()
    print("spinal hd,hd95,assd,asd,sp_ravd: ")
    print("spinal: ",sp_hd,sp_hd95,sp_assd,sp_asd,sp_ravd )
    print()
    print("spinal sp_volume_correlation,sp_volume_change_correlation: ")
    print("spinal: ",sp_volume_correlation, sp_volume_change_correlation)
    print()
    print("spinal sp_obj_assd,sp_obj_asd,sp_obj_tpr,sp_obj_fpr: ")
    print("spinal: ",sp_obj_assd,sp_obj_asd,sp_obj_tpr,sp_obj_fpr)
    print()
    print()




    print("MS Dice,Jaccard, precision,recall,ppv,sensitivity,specificity: ")
    print("MS", MS_dice,MS_jaccard, MS_precision, MS_recall,MS_ppv,MS_sensitivity,MS_specificity)
    print()
    print("MS true_negative_rate, true_positive_rate,positive_predictive_value: ")
    print("MS",MS_true_negative_rate,MS_true_positive_rate,MS_positive_predictive_value)
    print()
    print("MS hd,hd95,assd,asd,ravd: ")
    print("MS: ",MS_hd,MS_hd95,MS_assd,MS_asd,MS_ravd )
    print()
    print("MS MS_volume_correlation,MS_volume_change_correlation: ")
    print("MS: ",MS_volume_correlation, MS_volume_change_correlation)
    print()
    print("MS obj_assd,obj_asd,obj_tpr,obj_fpr: ")
    print("MS: ",MS_obj_assd,MS_obj_asd,MS_obj_tpr,MS_obj_fpr)
    print()
    print()

    print("all Dice,Jaccard, precision,recall,ppv,sensitivity,specificity: ")
    print("all:", all_dice,all_jaccard, all_precision, all_recall,all_ppv,all_sensitivity,all_specificity)
    print()
    print("all true_negative_rate, true_positive_rate,positive_predictive_value: ")
    print("all:",all_true_negative_rate,all_true_positive_rate,all_positive_predictive_value)
    print()
    print("all hd,hd95,assd,asd,ravd: ")
    print("all: ",all_hd,all_hd95,all_assd,all_asd,all_ravd )
    print()
    print("all MS_volume_correlation,MS_volume_change_correlation: ")
    print("all: ",all_volume_correlation, all_volume_change_correlation)
    print()
    print("all obj_assd,obj_asd,obj_tpr,obj_fpr: ")
    print("all: ",all_obj_assd,all_obj_asd,all_obj_tpr,all_obj_fpr)
    print()
    print()



print()
print("ortalama only spinal metrik hesaplama")
print("-----------------------------")
print()
print("spinal Dice,Jaccard, precision,recall,ppv,sensitivity,specificity: ")
print("spinal", toplam_sp_dice/len(X_test),toplam_sp_jaccard/len(X_test), toplam_sp_precision/len(X_test), toplam_sp_recall/len(X_test),toplam_sp_ppv/len(X_test),toplam_sp_sensitivity/len(X_test),toplam_sp_specificity/len(X_test))
print()
print("spinal true_negative_rate, true_positive_rate,sp_positive_predictive_value: ")
print("spinal",toplam_sp_true_negative_rate/len(X_test),toplam_sp_true_positive_rate/len(X_test),toplam_sp_positive_predictive_value/len(X_test))
print()
print("spinal hd,hd95,assd,asd,sp_ravd: ")
print("spinal: ",toplam_sp_hd/len(X_test),toplam_sp_hd95/len(X_test),toplam_sp_assd/len(X_test),toplam_sp_asd/len(X_test),toplam_sp_ravd/len(X_test) )
print()
print("spinal sp_volume_correlation,sp_volume_change_correlation: ")
print("spinal: ",toplam_sp_volume_correlation/len(X_test), toplam_sp_volume_change_correlation/len(X_test))
print()
print("spinal sp_obj_assd,sp_obj_asd,sp_obj_tpr,sp_obj_fpr: ")
print("spinal: ",toplam_sp_obj_assd/len(X_test),toplam_sp_obj_asd/len(X_test),toplam_sp_obj_tpr/len(X_test),toplam_sp_obj_fpr/len(X_test))
print()
print()


print("ortalama ms  metrik hesaplama")
print("-----------------------------")
print("MS Dice,Jaccard, precision,recall,ppv,sensitivity,specificity: ")
print("MS" ,toplam_MS_dice/len(X_test),toplam_MS_jaccard/len(X_test), toplam_MS_precision/len(X_test), toplam_MS_recall/len(X_test),toplam_MS_ppv/len(X_test),toplam_MS_sensitivity/len(X_test),toplam_MS_specificity/len(X_test))
print()
print("MS true_negative_rate, true_positive_rate,positive_predictive_value: ")
print("MS",toplam_MS_true_negative_rate/len(X_test),toplam_MS_true_positive_rate/len(X_test),toplam_MS_positive_predictive_value/len(X_test))
print()
print("MS hd,hd95,assd,asd,ravd: ")
print("MS: ",toplam_MS_hd/len(X_test),toplam_MS_hd95/len(X_test),toplam_MS_assd/len(X_test),toplam_MS_asd/len(X_test),toplam_MS_ravd/len(X_test))
print()
print("MS MS_volume_correlation,MS_volume_change_correlation: ")
print("MS: ",toplam_MS_volume_correlation/len(X_test), toplam_MS_volume_change_correlation/len(X_test))
print()
print("MS obj_assd,obj_asd,obj_tpr,obj_fpr: ")
print("MS: ",toplam_MS_obj_assd/len(X_test),toplam_MS_obj_asd/len(X_test),toplam_MS_obj_tpr/len(X_test),toplam_MS_obj_fpr/len(X_test))
print()
print()

print("ortalama all spinal metrik hesaplama")
print("-----------------------------")
print("all Dice,Jaccard, precision,recall,ppv,sensitivity,specificity: ")
print("all:", toplam_all_dice/len(X_test),toplam_all_jaccard/len(X_test), toplam_all_precision/len(X_test), toplam_all_recall/len(X_test),toplam_all_ppv/len(X_test),toplam_all_sensitivity/len(X_test),toplam_all_specificity/len(X_test))
print()
print("all true_negative_rate, true_positive_rate,positive_predictive_value: ")
print("all:",toplam_all_true_negative_rate/len(X_test),toplam_all_true_positive_rate/len(X_test),toplam_all_positive_predictive_value/len(X_test))
print()
print("all hd,hd95,assd,asd,ravd: ")
print("all: ",toplam_all_hd/len(X_test),toplam_all_hd95/len(X_test),toplam_all_assd/len(X_test),toplam_all_asd/len(X_test),toplam_all_ravd/len(X_test))
print()
print("all MS_volume_correlation,MS_volume_change_correlation: ")
print("all: ",toplam_all_volume_correlation/len(X_test), toplam_all_volume_change_correlation/len(X_test))
print()
print("all obj_assd,obj_asd,obj_tpr,obj_fpr: ")
print("all: ",toplam_all_obj_assd/len(X_test),toplam_all_obj_asd/len(X_test),toplam_all_obj_tpr/len(X_test),toplam_all_obj_fpr/len(X_test))
print()
print()




NameError: name 'unet_model' is not defined